In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calcular_admitancia(n_barras, lineas):
    Y = np.zeros((n_barras, n_barras), dtype=complex)
    for linea in lineas:
        i, j, r, x, b = linea
        Z = complex(r, x)
        Y[i-1, j-1] = -1/Z
        Y[j-1, i-1] = Y[i-1, j-1]
    
    for k in range(n_barras):
        Y[k, k] = -np.sum(Y[k, :])
    
    return Y

def flujo_carga_newton_raphson(n_barras, lineas, datos_barras, tol=1e-6, max_iter=20):
    Y = calcular_admitancia(n_barras, lineas)
    P = datos_barras[:, 1]  # Potencia activa especificada (P)
    Q = datos_barras[:, 2]  # Potencia reactiva especificada (Q)
    V = datos_barras[:, 3]  # Voltajes especificados o iniciales (V)
    ang = datos_barras[:, 4]  # Ángulos especificados o iniciales (Theta)

    PQ_indices = np.where(datos_barras[:, 0] == 1)[0]  # Indices de barras PQ
    PV_indices = np.where(datos_barras[:, 0] == 2)[0]  # Indices de barras PV
    slack_index = np.where(datos_barras[:, 0] == 3)[0][0]  # Índice de la barra slack

    n_PQ = len(PQ_indices)
    n_PV = len(PV_indices)

    # Newton-Raphson Iteration
    for iteration in range(max_iter):
        # Calculamos las potencias especificadas
        P_calc = np.zeros(n_barras)
        Q_calc = np.zeros(n_barras)
        
        for i in range(n_barras):
            for k in range(n_barras):
                if i != k:
                    P_calc[i] += V[i] * V[k] * (Y[i, k].real * np.cos(ang[i] - ang[k]) + Y[i, k].imag * np.sin(ang[i] - ang[k]))
                    Q_calc[i] -= V[i] * V[k] * (Y[i, k].real * np.sin(ang[i] - ang[k]) - Y[i, k].imag * np.cos(ang[i] - ang[k]))
            P_calc[i] += V[i]**2 * Y[i, i].real
            Q_calc[i] -= V[i]**2 * Y[i, i].imag

        # Desviaciones de potencia
        dP = P - P_calc
        dQ = Q - Q_calc

        # Construcción de la matriz Jacobiana
        J = np.zeros((2*n_PQ + n_PV, 2*n_PQ + n_PV))
        
        # Submatriz J1 (Derivadas de P respecto a Theta)
        for i in range(n_PQ + n_PV):
            for k in range(n_PQ + n_PV):
                if i == k:
                    J[i, k] = -Q_calc[PQ_indices[i]] - V[PQ_indices[i]]**2 * Y[PQ_indices[i], PQ_indices[i]].imag
                else:
                    J[i, k] = V[PQ_indices[i]] * V[PQ_indices[k]] * (Y[PQ_indices[i], PQ_indices[k]].real * np.sin(ang[PQ_indices[i]] - ang[PQ_indices[k]]) - Y[PQ_indices[i], PQ_indices[k]].imag * np.cos(ang[PQ_indices[i]] - ang[PQ_indices[k]]))

        # Submatriz J2 (Derivadas de P respecto a V)
        for i in range(n_PQ + n_PV):
            for k in range(n_PQ):
                if PQ_indices[i] == PQ_indices[k]:
                    J[i, n_PQ + n_PV + k] = V[PQ_indices[i]] * P_calc[PQ_indices[i]]
                else:
                    J[i, n_PQ + n_PV + k] = V[PQ_indices[i]] * (Y[PQ_indices[i], PQ_indices[k]].real * np.cos(ang[PQ_indices[i]] - ang[PQ_indices[k]]) + Y[PQ_indices[i], PQ_indices[k]].imag * np.sin(ang[PQ_indices[i]] - ang[PQ_indices[k]]))

        # Submatriz J3 (Derivadas de Q respecto a Theta)
        for i in range(n_PQ):
            for k in range(n_PQ + n_PV):
                if PQ_indices[i] == PQ_indices[k]:
                    J[n_PQ + n_PV + i, k] = V[PQ_indices[i]] * Q_calc[PQ_indices[i]]
                else:
                    J[n_PQ + n_PV + i, k] = -V[PQ_indices[i]] * V[PQ_indices[k]] * (Y[PQ_indices[i], PQ_indices[k]].real * np.cos(ang[PQ_indices[i]] - ang[PQ_indices[k]]) + Y[PQ_indices[i], PQ_indices[k]].imag * np.sin(ang[PQ_indices[i]] - ang[PQ_indices[k]]))

        # Submatriz J4 (Derivadas de Q respecto a V)
        for i in range(n_PQ):
            for k in range(n_PQ):
                if i == k:
                    J[n_PQ + n_PV + i, n_PQ + n_PV + k] = -P_calc[PQ_indices[i]] + V[PQ_indices[i]]**2 * Y[PQ_indices[i], PQ_indices[i]].real
                else:
                    J[n_PQ + n_PV + i, n_PQ + n_PV + k] = -V[PQ_indices[i]] * (Y[PQ_indices[i], PQ_indices[k]].real * np.sin(ang[PQ_indices[i]] - ang[PQ_indices[k]]) - Y[PQ_indices[i], PQ_indices[k]].imag * np.cos(ang[PQ_indices[i]] - ang[PQ_indices[k]]))

        # Vector de desviaciones
        dPQ = np.concatenate((dP[PQ_indices], dQ[PQ_indices]))

        # Solución del sistema lineal
        dx = np.linalg.solve(J, dPQ)

        # Actualización de ángulos y tensiones
        ang[PQ_indices] += dx[:n_PQ + n_PV]
        V[PQ_indices] += dx[n_PQ + n_PV:]

        # Verificación de convergencia
        if np.max(np.abs(dx)) < tol:
            break

    return V, ang

#Definición de las líneas de transmisión
# Cada línea está definida por:
# [Barra de origen (i), Barra de destino (j), Resistencia (pu), Reactancia (pu), Susceptancia (pu)]
n_barras = 6
lineas = [
    [1, 2, 0.02, 0.04, 0.01],
    [2, 3, 0.01, 0.03, 0.01],
    [3, 4, 0.02, 0.05, 0.01],
    [4, 5, 0.01, 0.03, 0.01],
    [5, 6, 0.03, 0.06, 0.01]
]
# Definición de los datos de las barras
# Cada fila representa una barra con los siguientes datos:
# [Tipo de barra, P especificada (pu), Q especificada (pu), Voltaje especificado (pu), Ángulo especificado (radianes)]
datos_barras = np.array([
    [3, 0.0, 0.0, 1.06, 0.0],  # Barra 1 (slack)
    [2, -1.0, 0.0, 1.0, 0.0],  # Barra 2 (PV)
    [1, -1.5, -0.5, 1.0, 0.0],  # Barra 3 (PQ)
    [1, -0.6, -0.2, 1.0, 0.0],  # Barra 4 (PQ)
    [1, -0.8, -0.3, 1.0, 0.0],  # Barra 5 (PQ)
    [1, -0.9, -0.4, 1.0, 0.0]   # Barra 6 (PQ)
])

# Resolver flujo de carga
V, ang = flujo_carga_newton_raphson(n_barras, lineas, datos_barras)

# Mostrar resultados
for i in range(n_barras):
    print(f'Barra {i+1}: V = {V[i]:.4f} pu, Theta = {ang[i]:.4f} rad')

# Graficar tensiones
plt.figure()
plt.bar(range(1, n_barras+1), V)
plt.xlabel('Barra')
plt.ylabel('Tensión (pu)')
plt.title('Tensiones en las barras')
plt.grid(True)
plt.show()

# Graficar ángulos
plt.figure()
plt.bar(range(1, n_barras+1), ang)
plt.xlabel('Barra')
plt.ylabel('Ángulo (rad)')
plt.title('Ángulos en las barras')
plt.grid(True)
plt.show()

# Calcular corrientes de línea
I_lines = []
for linea in lineas:
    i, j, r, x, _ = linea
    Z = complex(r, x)
    V_i = V[i-1] * (np.cos(ang[i-1]) + 1j*np.sin(ang[i-1]))
    V_j = V[j-1] * (np.cos(ang[j-1]) + 1j*np.sin(ang[j-1]))
    I_line = (V_i - V_j) / Z
    I_lines.append(np.abs(I_line))

# Graficar corrientes de línea
plt.figure()
plt.bar(range(1, len(I_lines)+1), I_lines)
plt.xlabel('Línea')
plt.ylabel('Corriente (pu)')
plt.title('Corrientes en las líneas')
plt.grid(True)
plt.show()



In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandapower as pp
import math

# Definimos los parámetros
f = 50  # frecuencia del sistema en [Hz]

Vg_LL = 110  # voltaje de gen [kV]
Vg_LL2 = 220 # Voltaje de barras
Imax = 0.457  # corriente máxima en las líneas [kA]

# Respecto a la carga
P = 150  # potencia activa en [MW]
Q = 100  # potencia reactiva en [Mvar]

# Crear red eléctrica
net = pp.create_empty_network()

# Añadir barras
bus1 = pp.create_bus(net, vn_kv=Vg_LL, name="Barra 1", slack=True)  # Barra 1 es designada como slack
bus2 = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 2")  # Barra 2 se conecta la carga
bus1A = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 1A")  # Barra 1A se conecta la carga
bus2A = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 2A")  # Barra 2A se conecta la carga
bus3A = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 3A")  # Barra 3A se conecta la carga
bus2B = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 2B")  # Barra 2B se conecta la carga
bus1B = pp.create_bus(net, vn_kv=Vg_LL2, name="Barra 1B")  # Barra 1B se conecta la carga

# Añadir líneas
pp.create_line(net, from_bus=bus2, to_bus=bus1A, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus1A, to_bus=bus2A, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus2A, to_bus=bus3A, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus3A, to_bus=bus2B, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus2B, to_bus=bus1B, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)
pp.create_line(net, from_bus=bus1B, to_bus=bus2, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", in_service=True)

# Añadir carga
pp.create_load(net, bus=bus1A, p_mw=0.2*P, q_mvar=0.2*Q)
pp.create_load(net, bus=bus2A, p_mw=0.35*P, q_mvar=0.35*Q)
pp.create_load(net, bus=bus3A, p_mw=0.15*P, q_mvar=0.15*Q)
pp.create_load(net, bus=bus2B, p_mw=0.6*P, q_mvar=0.6*Q)
pp.create_load(net, bus=bus1B, p_mw=0.1*P, q_mvar=0.1*Q)

# Se añade transformador de 100 MVA 220/110 kV entre la barra 1 y la barra 2
pp.create_transformer(net, hv_bus=bus2, lv_bus=bus1, name="trafo1", std_type="100 MVA 220/110 kV")

# Se añade fuente de generación conectada en la barra 1, (1[pu], el voltaje nominal [kV] se define en la barra a la que está conectada)
pp.create_ext_grid(net, bus=bus1, vm_pu=1, va_degree=0.0)

# Contador de iteraciones
iteraciones = 0

# Ejecutar el flujo de carga
success = False
while not success:
    iteraciones += 1
    pp.runpp(net, tolerance_mva=1e-8, max_iteration=1000)  # No es necesario ajustar max_iteration
    
    # Verificar si el flujo de carga convergió
    if net.converged:
        success = True

# Obtener resultados de las tensiones y ángulos en las barras
voltajes = net.res_bus.vm_pu.values  # Voltajes en p.u.
angulos = net.res_bus.va_degree.values  # Ángulos en grados

# Imprimir los resultados
for i in range(len(net.bus)):
    nombre = net.bus.name[i]
    voltaje = voltajes[i]
    angulo = angulos[i]
    print(f'Barra {nombre}: Voltaje = {voltaje:.4f} p.u., Ángulo = {angulo:.2f} grados')

print(f'\nNúmero de iteraciones: {iteraciones}')
print(f'Tolerancia utilizada: {1e-8:.2e} MVA')  # Mostramos la tolerancia utilizada, que es la especificada

Barra Barra 1: Voltaje = 1.0000 p.u., Ángulo = 0.00 grados
Barra Barra 2: Voltaje = 1.0334 p.u., Ángulo = -14.28 grados
Barra Barra 1A: Voltaje = 1.0315 p.u., Ángulo = -14.49 grados
Barra Barra 2A: Voltaje = 1.0292 p.u., Ángulo = -14.73 grados
Barra Barra 3A: Voltaje = 1.0284 p.u., Ángulo = -14.86 grados
Barra Barra 2B: Voltaje = 1.0274 p.u., Ángulo = -14.87 grados
Barra Barra 1B: Voltaje = 1.0324 p.u., Ángulo = -14.47 grados

Número de iteraciones: 1
Tolerancia utilizada: 1.00e-08 MVA
